# Install stuff

In [ ]:
import tensorflow as tf 
print(tf.__version__)

2.9.2


In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
!pip install malaya

# Import stuff

In [ ]:
%%time

import malaya
import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
import numpy as np
import random 
import json
import nltk
from nltk.stem import WordNetLemmatizer
import pickle

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

**Add intents3.json file**

JSON file display see what's inside

In [ ]:
with open('intents3.json') as file:
  data = json.load(file)

In [ ]:
print(data["intents"])

[{'tag': 'error', 'patterns': [''], 'responses': ['Maaf, saya masih tidak faham, sila cuba lagi. Untuk carian simptom, anda boleh masukkan kata kunci seperti batuk, sesak nafas, letih. Untuk carian pencegahan, anda boleh masukkan kata kunci seperti elak, ubat, cegah']}, {'tag': 'return', 'patterns': [''], 'responses': ["Hi, selamat datang ke Farmasi NLP!\n1. Taip 'mic'--untuk guna mic\n2. Masukkan simptom\n3. Pertanyaan tentang pencegahan penyakit\n4. Taip 'balik' untuk mula semula\n5. Taip 'keluar'--tamat chatbot"]}, {'tag': 'greetings', 'patterns': ['helo', 'hey', 'hi', 'hai', 'salam', 'apa khabar?', 'Bagaimana keadaannya'], 'responses': ['helo', 'hey!', 'Apa yang boleh saya buat untuk anda?']}, {'tag': 'goodbye', 'patterns': ['jumpa lagi', 'Selamat tinggal', 'cao', 'Semoga hari anda indah'], 'responses': ['Semoga hari anda indah', 'Selamat tinggal']}, {'tag': 'age', 'patterns': ['berapa umur', 'berapakah umur kamu?', 'berapakah umur anda', 'berapakah umur kamu', 'umur?'], 'responses

Forming empty list 

In [ ]:
words = []
labels = []
docs = []
ignore_words = ['!','”','#','$','%','&','’','(',')','*','+',',','-','.','/',':',';','<','=','>','?','@','[',']','^','_',"'",'{','|','}','~']

Looping the json file

In [ ]:
for intent in data['intents']:
  for pattern in intent['patterns']:

    #tokenize each word 
    # for malay
    tokenizer = malaya.tokenizer.Tokenizer()
    w= tokenizer.tokenize(pattern)
    # w = nltk.word_tokenize(pattern)
    words.extend(w)

    # add document in the corpus
    docs.append((w, intent['tag']))

    # add to label list
    if intent['tag'] not in labels:
      labels.append(intent['tag'])

The Stemming 

In [ ]:
pip install PySastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 210 kB 13.7 MB/s 


In [ ]:

# for malay
sastrawi = malaya.stem.sastrawi()

In [ ]:
# lemmatize each word
words = [sastrawi.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sorted labels 
labels = sorted(list(set(labels)))


# docs = combination of patterns and intents
print(len(docs),'documents')
# labels = intents
print(len(labels), 'labels',labels)
# words = all words, vocab
print(len(words),'unique lemmatized words', words)

173 documents
37 labels ['Abdonominal Pain', 'Abrasions', 'Animal bite', 'Bruises', 'Cuts', 'Diarrhea', 'Eye Injury', 'Fracture', 'Gastrointestinal Problems', 'Headache', 'Insect Bites', 'Nasal Congestion', 'Pulled Muscle', 'Rash', 'Skin Problems', 'Snake bite', 'Splinter', 'Sprains', 'Strains', 'Wound', 'age', 'asthma prevention', 'asthma symptoms', 'cold prevention', 'cold symptoms', 'depression prevention', 'depression symptoms', 'diabetes prevention', 'diabetes symptoms', 'error', 'fever prevention', 'fever symptoms', 'goodbye', 'greetings', 'name', 'return', 'stings']
167 unique lemmatized words ['ada', 'air', 'ais', 'akan', 'alahan', 'alami', 'ambil', 'amnya', 'anda', 'anjing', 'apa', 'asa', 'asma', 'atau', 'awak', 'badan', 'bagaimana', 'baik', 'bantu', 'batuk', 'bawa', 'beberapa', 'beli', 'berapa', 'berat', 'beritahu', 'bersin', 'biasa', 'bimbang', 'birit', 'boleh', 'buang', 'cakap', 'cao', 'cara', 'cedera', 'cegah', 'cirit', 'dada', 'dapat', 'dari', 'daripada', 'dehidrasi', 'de

/usr/local/lib/python3.8/dist-packages/malaya/stem.py:55: FutureWarning: Possible nested set at position 3
  or re.findall(_expressions['ic'], word.lower())


In [ ]:
# save as pickle file
pickle.dump(words,open('words2.pkl','wb'))
pickle.dump(labels,open('labels2.pkl','wb'))

# bag of words in one-hot encoding form
frequency

```
[0,0,0,0,0,0]
```



In [ ]:
# Create training data
training = []
# create an empty array for output
output_empty = [0] * len(labels)

# trainning set, bag of words for each sentence
for d in docs:
  # initialize bag of words
  bag=[]
  # list of tokenized words for the pattern
  pattern_words = d[0]
  # lemmatize each word -create base word, in attempt to represent related words
  pattern_words = [sastrawi.stem(word.lower()) for word in pattern_words]

  # create bag of words array with 1, if word match found in current pattern
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

  # output is a '0' for each tag and '1' for current tag (for each pattern)
  output_row = list(output_empty)
  output_row[labels.index(d[1])] = 1
  training.append([bag, output_row])

# shuffle the features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X--> patterns, Y--> intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


<ipython-input-18-3ace0aaa6d08>:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


# Modeling

In [ ]:
# create model --> 3 layers
# first layer - 128 neurons
# second layer - 64 neurons
# third output layer- neurons = number of intents to predict (softmax)

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# compile model.
# Stochastic gradient descent with Nesterov 
# accelerated gradient gives good results for this model
sgd = SGD(lr= 0.01, decay= 1e-6,momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


# fitting and saving the model
hist= model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
# model.save('refer_from_online_model.h5', hist)
model.save('malay_model_test.h5', hist)

print("model created")

Epoch 1/200


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


35/35 [==============================] - 1s 2ms/step - loss: 3.6249 - accuracy: 0.0231
Epoch 2/200
35/35 [==============================] - 0s 1ms/step - loss: 3.5453 - accuracy: 0.0405
Epoch 3/200
35/35 [==============================] - 0s 2ms/step - loss: 3.4934 - accuracy: 0.0462
Epoch 4/200
35/35 [==============================] - 0s 1ms/step - loss: 3.4296 - accuracy: 0.0925
Epoch 5/200
35/35 [==============================] - 0s 1ms/step - loss: 3.3430 - accuracy: 0.1503
Epoch 6/200
35/35 [==============================] - 0s 1ms/step - loss: 3.1681 - accuracy: 0.1676
Epoch 7/200
35/35 [==============================] - 0s 1ms/step - loss: 3.0355 - accuracy: 0.2254
Epoch 8/200
35/35 [==============================] - 0s 2ms/step - loss: 2.9089 - accuracy: 0.1908
Epoch 9/200
35/35 [==============================] - 0s 2ms/step - loss: 2.8075 - accuracy: 0.2370
Epoch 10/200
35/35 [==============================] - 0s 1ms/step - loss: 2.4913 - accuracy: 0.3121
Epoch 11/200
35/35 [=